In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils import data

import scipy.io as scio
import numpy as np

## Initialize Hyper-parameters

In [2]:
input_size = 11 # low dimension resolution
hidden_size = 50 #
num_classes = 10
num_epochs = 30
batch_size = 10
learning_rate = 0.001

## Feedforward Neural Network Model Structure

In [3]:
class ShallowNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ShallowNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size, num_classes)

    def forward(self,x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        return out

In [4]:
# Creating TrainLoader
# load low_dim training data
low_dim_data = scio.loadmat('low_dim_6000.mat')
low_dim = low_dim_data['low_dim'].reshape(6000)
low_dim = np.array([i.reshape(11) for i in low_dim])
low_dim -= low_dim.min()
low_dim /= low_dim.max()
# load labels
orig_data = scio.loadmat('subset_6000.mat')
labels = orig_data['subset_label_X']
labels = labels.reshape(6000)
# convert to dataLoader

tensor_low_dim = torch.Tensor(low_dim)#[:5000,:])
tensor_labels = torch.Tensor(labels)#[:5000])
low_dim_dataset = data.TensorDataset(tensor_low_dim, tensor_labels)
train_loader = data.DataLoader(dataset=low_dim_dataset,
                              batch_size=batch_size,)
print(tensor_low_dim.shape)



###### load test dataset #####

test_data = scio.loadmat('low_dim_orig_test.mat')
test_dat = test_data['low_dim'].reshape(10000)
test_dat = np.array([i.reshape(18) for i in test_dat])
test_dat = np.array([i[0].reshape(11) for i in test_dat])
test_dat -= test_dat.min()
test_dat /= test_dat.max()
test_labels = np.load('test_labels.npy')
print(len(test_dat))
# custom_train
train_dat = torch.Tensor(test_dat[:9000,:])
train_labels = torch.Tensor(test_labels[:9000])
train_custom_dataset = data.TensorDataset(train_dat, train_labels)
custom_train_loader = data.DataLoader(dataset=train_custom_dataset,
                              batch_size=batch_size,)

# custom_test

custom_test_dat = torch.Tensor(test_dat[9000:,:])
custom_test_labels = torch.Tensor(test_labels[9000:])
test_custom_dataset = data.TensorDataset(custom_test_dat, custom_test_labels)
custom_test_loader = data.DataLoader(dataset=test_custom_dataset,
                              batch_size=batch_size,)

torch.Size([6000, 11])
10000


In [5]:
len(custom_test_loader.dataset)

1000

In [6]:
# Creating TestLoader
# load_low_dim test data
low_dim_data = scio.loadmat('low_dim_test_1000.mat')
low_dim = low_dim_data['low_dim_test'].reshape(1000)
low_dim = np.array([i.reshape(11) for i in low_dim])
low_dim -= low_dim.min()
low_dim /= low_dim.max()
# load labels
labels = low_dim_data['subset_label_X_test']
labels = labels.reshape(1000)
# # convert to dataLoader
tensor_low_dim = torch.Tensor(low_dim[:,:])
tensor_labels = torch.Tensor(labels[:])
low_dim_dataset = data.TensorDataset(tensor_low_dim, tensor_labels)
test_loader = data.DataLoader(dataset=low_dim_dataset,
                              batch_size=batch_size,)
print(tensor_low_dim.shape)


###### load adv test dataset #####

adv_data = scio.loadmat('low_dim_adv_test.mat')
adv_dat = adv_data['low_dim'].reshape(10000)
adv_dat = np.array([i.reshape(18) for i in adv_dat])
adv_dat = np.array([i[0].reshape(11) for i in adv_dat])
adv_dat -= adv_dat.min()
adv_dat /= adv_dat.max()
adv_labels = np.load('test_labels.npy')
# custom_test
adv_dat = torch.Tensor(adv_dat[9000:,:])
adv_labels = torch.Tensor(adv_labels[9000:])
test_adv_dataset = data.TensorDataset(adv_dat, adv_labels)
custom_adv_loader = data.DataLoader(dataset=test_adv_dataset,
                              batch_size=batch_size,)

torch.Size([1000, 11])


In [7]:
low_dim_data['low_dim_test'][0,0]

array([[  15.288879],
       [  24.870827],
       [-295.22333 ],
       [ 290.0678  ],
       [-181.30225 ],
       [  66.25981 ],
       [ 617.94574 ],
       [-135.3153  ],
       [-323.53192 ],
       [ 475.9786  ],
       [  58.22062 ]], dtype=float32)

## Instantiate the NN

In [8]:
net = ShallowNetwork(input_size, hidden_size, num_classes)

## Enable GPU 

In [218]:
# net.cuda()    # You can comment out this line to disable GPU

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

## Train the Model

In [10]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(custom_train_loader):  
        images = Variable(images)         
        labels = Variable(labels.long())
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(images)                           # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, 6000//batch_size, loss.data))

Epoch [1/30], Step [100/600], Loss: 2.2663
Epoch [1/30], Step [200/600], Loss: 2.2413
Epoch [1/30], Step [300/600], Loss: 1.6579
Epoch [1/30], Step [400/600], Loss: 1.6944
Epoch [1/30], Step [500/600], Loss: 1.1110
Epoch [1/30], Step [600/600], Loss: 1.3602
Epoch [1/30], Step [700/600], Loss: 1.5869
Epoch [1/30], Step [800/600], Loss: 1.4300
Epoch [1/30], Step [900/600], Loss: 1.1025
Epoch [2/30], Step [100/600], Loss: 1.1228
Epoch [2/30], Step [200/600], Loss: 1.7108
Epoch [2/30], Step [300/600], Loss: 0.8638
Epoch [2/30], Step [400/600], Loss: 1.0836
Epoch [2/30], Step [500/600], Loss: 0.6665
Epoch [2/30], Step [600/600], Loss: 0.8907
Epoch [2/30], Step [700/600], Loss: 1.4223
Epoch [2/30], Step [800/600], Loss: 1.2303
Epoch [2/30], Step [900/600], Loss: 0.7937
Epoch [3/30], Step [100/600], Loss: 0.9497
Epoch [3/30], Step [200/600], Loss: 1.5360
Epoch [3/30], Step [300/600], Loss: 0.7206
Epoch [3/30], Step [400/600], Loss: 0.9724
Epoch [3/30], Step [500/600], Loss: 0.5413
Epoch [3/30

Epoch [22/30], Step [100/600], Loss: 0.6330
Epoch [22/30], Step [200/600], Loss: 0.9866
Epoch [22/30], Step [300/600], Loss: 0.3630
Epoch [22/30], Step [400/600], Loss: 0.3882
Epoch [22/30], Step [500/600], Loss: 0.0907
Epoch [22/30], Step [600/600], Loss: 0.1197
Epoch [22/30], Step [700/600], Loss: 0.3689
Epoch [22/30], Step [800/600], Loss: 1.0984
Epoch [22/30], Step [900/600], Loss: 0.2076
Epoch [23/30], Step [100/600], Loss: 0.5880
Epoch [23/30], Step [200/600], Loss: 0.9851
Epoch [23/30], Step [300/600], Loss: 0.3616
Epoch [23/30], Step [400/600], Loss: 0.3897
Epoch [23/30], Step [500/600], Loss: 0.0920
Epoch [23/30], Step [600/600], Loss: 0.1245
Epoch [23/30], Step [700/600], Loss: 0.3639
Epoch [23/30], Step [800/600], Loss: 1.1154
Epoch [23/30], Step [900/600], Loss: 0.2020
Epoch [24/30], Step [100/600], Loss: 0.5418
Epoch [24/30], Step [200/600], Loss: 0.9741
Epoch [24/30], Step [300/600], Loss: 0.3223
Epoch [24/30], Step [400/600], Loss: 0.3609
Epoch [24/30], Step [500/600], L

## Test the Model

Similar to training the nerual network, we also need to load batches of test images and collect the outputs. The differences are that:
(1) No loss & weights calculation
(2) No wights update
(3) Has correct prediction calculation

In [14]:
correct = 0
total = 0
for images, labels in custom_test_loader:
    labels = labels.long()
    images = Variable(images)
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    correct += (predicted == labels).sum()     # Increment the correct count

print(labels)
print('Accuracy of the network on the 10K test images: %d %%' % (100 * float(correct) / float(total)))

tensor([7, 4, 8, 8, 0, 8, 5, 0, 7, 8])
Accuracy of the network on the 10K test images: 86 %


In [213]:
correct

tensor(868)

## Save the trained Model for future use

In [11]:
# Save the model
torch.save(net.state_dict(), 'fnn_model.pkl')

In [10]:
from tensorboardX import SummaryWriter
writer = SummaryWriter('runs/mnist')

In [11]:
imgs = None
for images, labels in test_loader:
    imgs = images
    break
writer.add_graph(net,tensor_low_dim)

In [127]:
writer.add_graph(net,tensor_low_dim)

In [1]:
writer.close()

NameError: name 'writer' is not defined

In [140]:
test_data = scio.loadmat('low_dim_orig_test.mat')
test_dat = test_data['low_dim'].reshape(10000)
test_dat = np.array([i.reshape(18) for i in test_dat])
test_dat = np.array([i[0].reshape(11) for i in test_dat])

In [108]:
test_dat.min()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [103]:
test_labels = np.load('test_labels.npy')

In [109]:
low_dim[0,:]

array([0.4279091 , 0.51963484, 0.52086675, 0.47780225, 0.5300925 ,
       0.50020856, 0.30898094, 0.41586205, 0.2276364 , 0.5086932 ,
       0.44525117], dtype=float32)

In [131]:
low_dim[0]

array([0.4279091 , 0.51963484, 0.52086675, 0.47780225, 0.5300925 ,
       0.50020856, 0.30898094, 0.41586205, 0.2276364 , 0.5086932 ,
       0.44525117], dtype=float32)

In [143]:
test_dat[0]

array([ 9.1296997e+00,  6.9629767e+02,  8.3496094e-02,  1.0404474e+02,
       -3.2623260e+02, -2.8722021e+02,  8.4440002e+00, -1.9635437e+02,
        5.7214197e+02,  1.8434105e+01,  1.3124892e+02], dtype=float32)